# Imports

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from controller import Controller

import pycaret.regression as pr
import pycaret.classification as pc

In [3]:
c = Controller('i02')

COLS_TO_DROP_BACKERS = [
    'PROJECT_ID',
    'DEADLINE',
    'STATE_CHANGED_AT',
    'CREATED_AT',
    'LAUNCHED_AT',
    'KEYWORDS',
    
    'DESC',
    'NAME',
]

# Load data

In [4]:
filepath = fr'{c.get_path_data_prepared()}/01_df_oot.pkl'
df_test = pd.read_pickle(filepath)
df_test

,PROJECT_ID,NAME,DESC,GOAL,KEYWORDS,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,DEADLINE,STATE_CHANGED_AT,CREATED_AT,LAUNCHED_AT,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS
0,kkst917493670,Br thair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08 23:59:45,2015-12-08 23:59:45,2015-10-28 03:23:01,2015-10-29 22:59:45,43.612222,3.798008,1004.612222,6.913352,961.000000
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22 04:07:00,2016-01-22 04:07:00,2015-12-16 20:22:03,2015-12-18 04:07:00,31.749167,3.488878,871.749167,6.771648,840.000000
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03 00:25:00,2016-01-03 00:25:00,2015-11-26 23:42:36,2015-11-28 00:55:28,25.214444,3.266311,888.706667,6.790892,863.492222
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16 18:51:58,2015-10-16 18:52:10,2015-08-30 20:33:50,2015-09-16 18:51:58,406.302222,6.009555,1126.302222,7.027583,720.000000
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01 02:14:00,2016-05-01 02:14:04,2016-02-18 03:14:36,2016-03-18 21:01:30,713.781667,6.571977,1750.990000,7.468508,1037.208333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63460,kkst538121766,Singing Whales And Other Tales,Aesop's Fables meets Beatrix Potter in the 43 ...,6000.0,singing-whales-and-other-tales,False,US,USD,2015-07-01 02:00:00,2015-07-01 02:00:12,2015-04-11 00:15:15,2015-05-08 20:16:33,668.021667,6.505816,1945.745833,7.573914,1277.724167
63461,kkst1927608005,The Dragon of Wawel Hill,A children's book based on an old legend of a ...,1500.0,the-dragon-of-wawel-hill,False,US,USD,2015-10-26 09:48:27,2015-10-26 09:48:28,2015-09-04 01:15:21,2015-09-11 09:48:27,176.551667,5.179262,1256.551667,7.136922,1080.000000
63462,kkst68821321,Someone You Love Has Cancer- A Childs Guide to...,"Providing a realistic picture book, that gentl...",4000.0,someone-you-love-has-cancer-a-childs-guide-to-...,False,CA,CAD,2016-06-01 15:39:00,2016-06-01 15:39:00,2016-04-25 15:27:58,2016-04-25 16:29:59,1.033611,0.709813,888.183889,6.790304,887.150278
63463,kkst665771257,A GERM'S JOURNEY,A GERM'S JOURNEY Traveling Soon to a Bookshelf...,42800.0,a-germs-journey,False,US,USD,2016-06-11 16:50:10,2016-06-11 16:50:10,2016-03-29 17:11:38,2016-04-12 16:50:10,335.642222,5.819021,1775.642222,7.482480,1440.000000


In [5]:
model_path_catboost = f'{c.get_path_iteration("i01")}/final_catboost'
model_catboost = pc.load_model(model_path_catboost)
model_catboost

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['GOAL', 'BACKERS_COUNT',
                                             'CREATE_LAUNCH_HOURS',
                                             'CREATE_LAUNCH_HOURS_LOG',
                                             'CREATE_DEADLINE_HOURS',
                                             'CREATE_DEADLINE_HOURS_LOG',
                                             'LAUNCHED_DEADLINE_HOURS'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['COUNTRY', 'CURRENCY'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['COUNTRY', 'CURRENCY'],
                                    transformer=OneHotEncoder(cols=['COUNTRY',
                                                                    'CURRENCY'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('trained_model',
                 <catboost.core.CatBoostClassifier object at 0x7fc2c5ed8130>)])

In [6]:
model_path_stack = f'{c.get_path_iteration("i02")}/final_stack_model'
model_stack = pr.load_model(model_path_stack)
model_stack

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['GOAL', 'CREATE_LAUNCH_HOURS',
                                             'CREATE_LAUNCH_HOURS_LOG',
                                             'CREATE_DEADLINE_HOURS',
                                             'CREATE_DEADLINE_HOURS_LOG',
                                             'LAUNCHED_DEADLINE_HOURS'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['COUNTRY', 'CURRENCY'],
                                    transfor...
                                                <catboost.core.CatBoostRegressor object at 0x7fc2c67998a0>),
                                               ('Gradient Boosting Regressor',
                                                GradientBoostingRegressor(learning_rate=0.05,
                                                                          max_depth=6,
                                                                          max_features='sqrt',
                                                                          min_impurity_decrease=0.3,
                                                                          min_samples_leaf=4,
                                                                          min_samples_split=10,
                                                                          n_estimators=270,
                                                                          random_state=42,
                                                                          subsample=0.7))],
                                   final_estimator=LinearRegression(n_jobs=-1),
                                   n_jobs=-1, passthrough=True))])

# Predict backers

In [7]:
df_test_backers = df_test.drop(COLS_TO_DROP_BACKERS, axis=1)
df_test_backers

,GOAL,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS
0,7000.0,False,US,USD,43.612222,3.798008,1004.612222,6.913352,961.000000
1,35000.0,False,US,USD,31.749167,3.488878,871.749167,6.771648,840.000000
2,49500.0,False,US,USD,25.214444,3.266311,888.706667,6.790892,863.492222
3,40000.0,False,US,USD,406.302222,6.009555,1126.302222,7.027583,720.000000
4,20000.0,False,US,USD,713.781667,6.571977,1750.990000,7.468508,1037.208333
...,...,...,...,...,...,...,...,...,...
63460,6000.0,False,US,USD,668.021667,6.505816,1945.745833,7.573914,1277.724167
63461,1500.0,False,US,USD,176.551667,5.179262,1256.551667,7.136922,1080.000000
63462,4000.0,False,CA,CAD,1.033611,0.709813,888.183889,6.790304,887.150278
63463,42800.0,False,US,USD,335.642222,5.819021,1775.642222,7.482480,1440.000000


In [8]:
df_catboost = pr.predict_model(model_stack, df_test)
df_catboost = df_catboost.rename(columns={'prediction_label': 'BACKERS_COUNT'})
df_catboost

,PROJECT_ID,NAME,DESC,GOAL,KEYWORDS,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,DEADLINE,STATE_CHANGED_AT,CREATED_AT,LAUNCHED_AT,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS,BACKERS_COUNT
0,kkst917493670,Br thair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08 23:59:45,2015-12-08 23:59:45,2015-10-28 03:23:01,2015-10-29 22:59:45,43.612221,3.798008,1004.612244,6.913352,961.000000,2.807424
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22 04:07:00,2016-01-22 04:07:00,2015-12-16 20:22:03,2015-12-18 04:07:00,31.749166,3.488878,871.749146,6.771648,840.000000,2.683242
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03 00:25:00,2016-01-03 00:25:00,2015-11-26 23:42:36,2015-11-28 00:55:28,25.214445,3.266311,888.706665,6.790892,863.492249,2.695426
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16 18:51:58,2015-10-16 18:52:10,2015-08-30 20:33:50,2015-09-16 18:51:58,406.302216,6.009555,1126.302246,7.027583,720.000000,3.628349
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01 02:14:00,2016-05-01 02:14:04,2016-02-18 03:14:36,2016-03-18 21:01:30,713.781677,6.571977,1750.989990,7.468508,1037.208374,4.118465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63460,kkst538121766,Singing Whales And Other Tales,Aesop's Fables meets Beatrix Potter in the 43 ...,6000.0,singing-whales-and-other-tales,False,US,USD,2015-07-01 02:00:00,2015-07-01 02:00:12,2015-04-11 00:15:15,2015-05-08 20:16:33,668.021667,6.505816,1945.745850,7.573915,1277.724121,3.129497
63461,kkst1927608005,The Dragon of Wawel Hill,A children's book based on an old legend of a ...,1500.0,the-dragon-of-wawel-hill,False,US,USD,2015-10-26 09:48:27,2015-10-26 09:48:28,2015-09-04 01:15:21,2015-09-11 09:48:27,176.551666,5.179262,1256.551636,7.136922,1080.000000,2.554801
63462,kkst68821321,Someone You Love Has Cancer- A Childs Guide to...,"Providing a realistic picture book, that gentl...",4000.0,someone-you-love-has-cancer-a-childs-guide-to-...,False,CA,CAD,2016-06-01 15:39:00,2016-06-01 15:39:00,2016-04-25 15:27:58,2016-04-25 16:29:59,1.033611,0.709813,888.183899,6.790304,887.150269,1.617021
63463,kkst665771257,A GERM'S JOURNEY,A GERM'S JOURNEY Traveling Soon to a Bookshelf...,42800.0,a-germs-journey,False,US,USD,2016-06-11 16:50:10,2016-06-11 16:50:10,2016-03-29 17:11:38,2016-04-12 16:50:10,335.642212,5.819021,1775.642212,7.482481,1440.000000,2.534764


# Predict final status

In [9]:
df_predictions = pc.predict_model(model_catboost, df_catboost)
df_predictions

,PROJECT_ID,NAME,DESC,GOAL,KEYWORDS,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,DEADLINE,STATE_CHANGED_AT,CREATED_AT,LAUNCHED_AT,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS,BACKERS_COUNT,prediction_label,prediction_score
0,kkst917493670,Br thair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08 23:59:45,2015-12-08 23:59:45,2015-10-28 03:23:01,2015-10-29 22:59:45,43.612221,3.798008,1004.612244,6.913352,961.000000,2.807424,0,0.9998
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22 04:07:00,2016-01-22 04:07:00,2015-12-16 20:22:03,2015-12-18 04:07:00,31.749166,3.488878,871.749146,6.771648,840.000000,2.683242,0,0.9999
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03 00:25:00,2016-01-03 00:25:00,2015-11-26 23:42:36,2015-11-28 00:55:28,25.214445,3.266311,888.706665,6.790892,863.492249,2.695426,0,0.9998
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16 18:51:58,2015-10-16 18:52:10,2015-08-30 20:33:50,2015-09-16 18:51:58,406.302216,6.009555,1126.302246,7.027583,720.000000,3.628349,0,0.9998
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01 02:14:00,2016-05-01 02:14:04,2016-02-18 03:14:36,2016-03-18 21:01:30,713.781677,6.571977,1750.989990,7.468508,1037.208374,4.118465,0,0.9998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63460,kkst538121766,Singing Whales And Other Tales,Aesop's Fables meets Beatrix Potter in the 43 ...,6000.0,singing-whales-and-other-tales,False,US,USD,2015-07-01 02:00:00,2015-07-01 02:00:12,2015-04-11 00:15:15,2015-05-08 20:16:33,668.021667,6.505816,1945.745850,7.573915,1277.724121,3.129497,0,0.9998
63461,kkst1927608005,The Dragon of Wawel Hill,A children's book based on an old legend of a ...,1500.0,the-dragon-of-wawel-hill,False,US,USD,2015-10-26 09:48:27,2015-10-26 09:48:28,2015-09-04 01:15:21,2015-09-11 09:48:27,176.551666,5.179262,1256.551636,7.136922,1080.000000,2.554801,0,0.9973
63462,kkst68821321,Someone You Love Has Cancer- A Childs Guide to...,"Providing a realistic picture book, that gentl...",4000.0,someone-you-love-has-cancer-a-childs-guide-to-...,False,CA,CAD,2016-06-01 15:39:00,2016-06-01 15:39:00,2016-04-25 15:27:58,2016-04-25 16:29:59,1.033611,0.709813,888.183899,6.790304,887.150269,1.617020,0,0.9997
63463,kkst665771257,A GERM'S JOURNEY,A GERM'S JOURNEY Traveling Soon to a Bookshelf...,42800.0,a-germs-journey,False,US,USD,2016-06-11 16:50:10,2016-06-11 16:50:10,2016-03-29 17:11:38,2016-04-12 16:50:10,335.642212,5.819021,1775.642212,7.482481,1440.000000,2.534764,0,0.9999


In [10]:
pd.concat([
    df_predictions['prediction_label'].value_counts(),
    df_predictions['prediction_label'].value_counts(normalize=True),
], axis=1)

,prediction_label,prediction_label
0,63355,0.998267
1,110,0.001733


# Saving to files

In [13]:
filename = f'{c.get_path_data_prepared()}/05_final_predictions_test_subset.pkl'
df_predictions.to_pickle(filename)